In [1]:
import pycollatinus

lemmatiseur = pycollatinus.lemmatiseur.Lemmatiseur()

## Étude sur l'utilisation de l'annotation morphologique

1. Alignement sur la base avec PyCollatinus sur la base (Cas+Nombre), récupération du genre quand possible.
2. Récupération dans le même temps des formes pour trouver des 

In [71]:
from collections import Counter, defaultdict

def to_list(line):
    return line.strip().split()

def collatinus_morph(morph_string: str):
    out = {}
    gend = None
    for info in morph_string.split():
        if info in ["nominatif", "accusatif", "ablatif", "génitif", "vocatif", "datif", "locatif"]:
            out["Case"] = info[0].upper()+info[1:3].replace("é", "e")
        elif info in ["singulier", "pluriel"]:
            out["Numb"] = info[0].upper()+info[1:4]
        elif info in ["masculin", "féminin", "neutre"]:
            gend = info[0].upper()+info[1:4]
    return out, gend


input_file = "../../LASLA/genders.tsv"
limit = None
possibilities = defaultdict(list)
forms = defaultdict(list)
with open(input_file) as f:
    for line_no, line in enumerate(f):
        if line_no == 0:
            header = to_list(line)
        elif line_no == limit:
            break
        else:
            line = dict(zip(header, to_list(line)))
            # On ajoute le dictionnaire de la forme liée à sa morphologie "simplifiée"
            forms[line["token"]].append({"Case": line["Case"], "Numb": line["Numb"], "Gend": line["Gend"], "lemma": line["lemma"]})
            # Ici, on s'occupe de ce qui va être aligné avec Collatinus
            if line["Gend"] not in ["_", "Masc", "Fem", "Neut"]:
                working = []
                for possibility in lemmatiseur.lemmatise(line["token"]):
                    morph, gend = collatinus_morph(possibility["morph"])
                    if morph == {"Case": line["Case"], "Numb": line["Numb"]}:
                        if gend:
                            working.append(gend)
                possibilities[line["Gend"]].append(len(set(working)))

                
dico = {g: Counter(val) for g, val in possibilities.items()}

### Alignement avec collatinus

Est aligné un terme dont la forme, le cas et le nombre sont identiques. Pour chacun de ces alignements, on compte le nombre de genres associés.

In [64]:
cats = sorted(list(set([value for file in dico.values() for value in file.keys()])))
print(",".join([""] + [cnt for cnt in dico]))
for key in cats:
    print(",".join([str(key)] + [str(dico[cnt].get(key, "0")) for cnt in dico]))


,MascNeut,MascFem,Com,Gend
0,783,653,1478,2
1,219,3863,33,0
2,65852,25155,2512,0
3,1591,172,67206,0


### Recherche d'instabilité dans l'annotation

Bien que peu probable, on vérifie que la doctrine d'annotation morphologique a été suivie en vérifiant le nombre de formes et de lemmes qui possèdent les deux annotations.

In [65]:
def map_val(dic):
    return ("|".join([key+"="+val for key, val in dic.items() if key!="Gend"]), dic["Gend"])
forms2 = {g: Counter(list(map(map_val, val))) for g, val in forms.items()}

In [70]:
# Formes annotées 
out = []
lemma = []
for form, counter in forms2.items():
    for (case_numb, gend), count in counter.items():
        if gend not in ["Masc", "Fem", "Neut"]:
            for secondary_gend in ["Masc", "Fem", "Neut"]:
                if (case_numb, secondary_gend) in counter:
                    out.append((gend, secondary_gend))
                    lemma.append(case_numb.split("|")[-1].split("=")[-1])

print(Counter(out))
print(sorted(list(set(lemma))))
print(len(set(lemma)))

Counter({('MascNeut', 'Neut'): 71, ('MascNeut', 'Masc'): 63, ('MascNeut', 'Fem'): 27, ('MascFem', 'Masc'): 26, ('Com', 'Masc'): 17, ('Com', 'Fem'): 17, ('Com', 'Neut'): 16, ('MascFem', 'Fem'): 15, ('MascFem', 'Neut'): 7})
['1', '10', '11', '13', '15', '2', '2000', '21', '22', '3', '32', '5', '6', '7', '8', 'achaemenius', 'admoueo', 'aduersus', 'aduncus', 'affero', 'ago', 'alacer', 'albus', 'alienus', 'aliquis', 'alter', 'ambo', 'amplus', 'anhelo', 'annosus', 'aonius', 'armatus', 'ascraeus', 'assumo', 'astutus', 'barbatus', 'bistonis', 'bonus', 'caelestis', 'campester', 'capitolinus', 'careo', 'carus', 'cecropis', 'celeber', 'celer', 'certus', 'ceterus', 'clamito', 'claudo', 'coepio', 'colligo', 'comis', 'comminiscor', 'concedo', 'confero', 'conflo', 'conscribo', 'consulo', 'cornipes', 'cous', 'credo', 'crinitus', 'danaus', 'decimus', 'dedo', 'deligo', 'dico', 'difficilis', 'dignor', 'disertus', 'disicio', 'diuus', 'do', 'dubius', 'duco', 'dulichius', 'ecquis', 'emax', 'eo', 'euboicus',